In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('dataset/train')
label_types = os.listdir('dataset/train')

dataset_path.pop(0)
label_types.pop(0)

print (label_types)  


['Autistic', 'Normal']


In [2]:
#Preparing Training data


In [3]:
rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('dataset/train' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('dataset/train' + '/' +item) + '/' + room))
    
# Build a dataframe        
train_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(train_df.head())
print(train_df.tail())

        tag                     video_name
0  Autistic  dataset/train/Autistic/18.mp4
1  Autistic  dataset/train/Autistic/19.mp4
2  Autistic  dataset/train/Autistic/20.mp4
3  Autistic  dataset/train/Autistic/21.mp4
4  Autistic  dataset/train/Autistic/22.mp4
       tag                  video_name
29  Normal  dataset/train/Normal/5.mp4
30  Normal  dataset/train/Normal/6.mp4
31  Normal  dataset/train/Normal/7.mp4
32  Normal  dataset/train/Normal/8.mp4
33  Normal  dataset/train/Normal/9.mp4


In [4]:
df = train_df.loc[:,['video_name','tag']]
df
df.to_csv('train.csv')

In [5]:
## Preparing Test Data

In [6]:
dataset_path = os.listdir('dataset/test')
dataset_path.pop(0)
label_types.pop(0)
print(dataset_path)

room_types = os.listdir('dataset/test')
print("Types of activities found: ", len(dataset_path))

rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('dataset/test' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('dataset/test' + '/' +item) + '/' + room))
    
# Build a dataframe        
test_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(test_df.head())
print(test_df.tail())

df = test_df.loc[:,['video_name','tag']]
df
df.to_csv('test.csv')


['Autistic', 'Normal']
Types of activities found:  2
        tag                    video_name
0  Autistic  dataset/test/Autistic/10.mp4
1  Autistic  dataset/test/Autistic/11.mp4
2  Autistic  dataset/test/Autistic/12.mp4
3  Autistic   dataset/test/Autistic/7.mp4
4  Autistic   dataset/test/Autistic/8.mp4
       tag                 video_name
7   Normal  dataset/test/Normal/2.mp4
8   Normal  dataset/test/Normal/3.mp4
9   Normal  dataset/test/Normal/4.mp4
10  Normal  dataset/test/Normal/5.mp4
11  Normal  dataset/test/Normal/6.mp4


In [7]:
!pip install tensorflow

In [10]:
!pip install git+https://github.com/tensorflow/docs

  Cloning https://github.com/tensorflow/docs to c:\users\asus\appdata\local\temp\pip-req-build-dury0rx7
  Resolved https://github.com/tensorflow/docs to commit 613618a5ce6f0acfd0d70e475bb6e46fae18c843


  Running command git clone -q https://github.com/tensorflow/docs 'C:\Users\asus\AppData\Local\Temp\pip-req-build-dury0rx7'


In [13]:
!pip install imutils

  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25872 sha256=c9635e0b86b6e588138faeae9344811c1b4ee062732d5d994b3611179c5a8169
  Stored in directory: c:\users\asus\appdata\local\pip\cache\wheels\4b\a5\2d\4a070a801d3a3d93f033d3ee9728f470f514826e89952df3ea
Successfully built imutils


In [15]:
!pip install opencv-python

  Using cached opencv_python-4.6.0.66-cp36-abi3-win_amd64.whl (35.6 MB)


In [16]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [17]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
  except RuntimeError as e:
    print(e)


In [18]:
## Data preparation

In [19]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")


train_df.sample(10)


Total videos for training: 34
Total videos for testing: 12


,Unnamed: 0,video_name,tag
21,21,dataset/train/Normal/13.mp4,Normal
7,7,dataset/train/Autistic/25.mp4,Autistic
5,5,dataset/train/Autistic/23.mp4,Autistic
1,1,dataset/train/Autistic/19.mp4,Autistic
3,3,dataset/train/Autistic/21.mp4,Autistic
28,28,dataset/train/Normal/4.mp4,Normal
29,29,dataset/train/Normal/5.mp4,Normal
26,26,dataset/train/Normal/2.mp4,Normal
12,12,dataset/train/Autistic/30.mp4,Autistic
0,0,dataset/train/Autistic/18.mp4,Autistic


In [20]:
## Feed the videos to a network:


In [21]:
IMG_SIZE = 224


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)



In [22]:
## Feature Extraction


In [23]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()


87910968/87910968 [==============================] - 55s 1us/step


In [24]:
## Label Encoding


In [25]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["tag"]))
print(label_processor.get_vocabulary())

labels = train_df["tag"].values
labels = label_processor(labels[..., None]).numpy()
labels


['Autistic', 'Normal']


array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]], dtype=int64)

In [26]:
#Define hyperparameters

IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 100

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048


In [27]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    
    ##take all classlabels from train_df column named 'tag' and store in labels
    labels = df["tag"].values
    
    #convert classlabels to label encoding
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool") # 145,20
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32") #145,20,2048

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")



print(f"train_labels in train set: {train_labels.shape}")

print(f"test_labels in train set: {test_labels.shape}")

# MAX_SEQ_LENGTH = 20, NUM_FEATURES = 2048. We have defined this above under hyper parameters


Frame features in train set: (34, 20, 2048)
Frame masks in train set: (34, 20)
train_labels in train set: (34, 1)
test_labels in train set: (12, 1)


In [28]:
## The sequence model


In [29]:
# Utility for our sequence model.
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

EPOCHS = 30
# Utility for running experiments.
def run_experiment():
    filepath = "./tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

Epoch 1/30
1/1 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.7391
Epoch 1: val_loss improved from inf to 0.69415, saving model to ./tmp\video_classifier
1/1 [==============================] - 23s 23s/step - loss: 0.6931 - accuracy: 0.7391 - val_loss: 0.6941 - val_accuracy: 0.0000e+00
Epoch 2/30
1/1 [==============================] - ETA: 0s - loss: 0.6927 - accuracy: 0.7391
Epoch 2: val_loss did not improve from 0.69415
1/1 [==============================] - 0s 100ms/step - loss: 0.6927 - accuracy: 0.7391 - val_loss: 0.6951 - val_accuracy: 0.0000e+00
Epoch 3/30
1/1 [==============================] - ETA: 0s - loss: 0.6922 - accuracy: 0.7391
Epoch 3: val_loss did not improve from 0.69415
1/1 [==============================] - 0s 97ms/step - loss: 0.6922 - accuracy: 0.7391 - val_loss: 0.6962 - val_accuracy: 0.0000e+00
Epoch 4/30
1/1 [==============================] - ETA: 0s - loss: 0.6917 - accuracy: 0.7391
Epoch 4: val_loss did not improve from 0.69415
1/1 [==

In [33]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames

test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")

test_frames = sequence_prediction(test_video)


Test video path: dataset/test/Normal/2.mp4
1/1 [==============================] - 0s 47ms/step
  Autistic: 50.05%
  Normal: 49.95%
